In [1]:
!pip install open_clip_torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.8 MB/s eta 0:00:00


In [29]:
import yaml
import os
config_data = {
    "paths": {
        "data_dir": "/kaggle/working/binclas",
        "log_dir": "logs",
        "cache_dir": "/kaggle/working/",
        "checkpoint_dir": "/kaggle/working/"
    },
    "wandb": {
        "using": False,
        "api_key": "your_wandb_api_key",
        "project": "project_name",
        "run_name_template": "{hidden_dim}x{num_hidden_layers}_training"
    },
    # Model: convnext_base_w, Pretrained: laion2b_s13b_b82k

    "training": {
        "batch_size":4 ,
        "num_epochs": 10,
        "accumulation_steps": 2
    },
    "model": {
        "name": "convnext_base_w", # Replace the Model with desired CLIP Model
        "pretrained": "laion2b_s13b_b82k", # Corresponding Pretrained Dataset
        "clip_dim": 640, # Don't forget to change this
        "hidden_dim": [256],
        "dropout_rate": [0.1],
        "num_hidden_layers": [1],
    },
    "optimizer": {
        "clip_lr": [1e-5],
        "predictor_lr": [5e-5],
        "weight_decay": [0.001],
        "beta1": [0.9],
        "beta2": [0.999]
    },
    "scheduler": {
        "gamma": 0.1,
        "milestones": [4, 6, 10]
    }
}

output_dir = "/kaggle/working/"  # Replace with the desired directory
file_name = "config.yml"

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)

# Full file path
file_path = os.path.join(output_dir, file_name)

# Write the YAML content to the file
with open(file_path, "w") as file:
    yaml.dump(config_data, file, default_flow_style=False)

print(f"YAML file saved to {file_path}")


YAML file saved to /kaggle/working/config.yml


In [3]:
import os
import random
import shutil

# Define folder paths
scratches_folder = "/kaggle/input/binnary-classification/scratches"
no_scratches_folder = "/kaggle/input/binnary-classification/no_scratches"
test_images_folder = "/kaggle/working/binclas"
remaining_images_folder = "/kaggle/working/testdata"

# Create test_images and testdata folders if they don't exist
os.makedirs(test_images_folder, exist_ok=True)
os.makedirs(remaining_images_folder, exist_ok=True)

# Function to copy random files from a directory while preserving structure
def copy_random_files_with_structure(source_folder, destination_folder, subfolder_name, percentage):
    # Create subfolder in the destination folder
    destination_subfolder = os.path.join(destination_folder, subfolder_name)
    os.makedirs(destination_subfolder, exist_ok=True)
    
    # Get all files in the source folder
    files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
    # Calculate the number of files to copy
    count = max(1, int(len(files) * percentage / 100))  # Ensure at least one file is selected if percentage > 0
    # Randomly select files
    selected_files = random.sample(files, min(count, len(files)))
    # Copy each selected file to the destination subfolder
    for file in selected_files:
        shutil.copy(os.path.join(source_folder, file), destination_subfolder)
    # Return remaining files
    remaining_files = set(files) - set(selected_files)
    return remaining_files

# Copy remaining files and preserve structure
def copy_remaining_files_with_structure(source_folder, destination_folder, remaining_files, subfolder_name):
    # Create subfolder in the destination folder
    destination_subfolder = os.path.join(destination_folder, subfolder_name)
    os.makedirs(destination_subfolder, exist_ok=True)
    
    # Copy each remaining file to the destination subfolder
    for file in remaining_files:
        shutil.copy(os.path.join(source_folder, file), destination_subfolder)

# Copy 10% of images and maintain structure for `binclas`
remaining_scratches = copy_random_files_with_structure(scratches_folder, test_images_folder, "scratches", 90)
remaining_no_scratches = copy_random_files_with_structure(no_scratches_folder, test_images_folder, "no_scratches", 90)

# Copy remaining images and maintain structure for `testdata`
copy_remaining_files_with_structure(scratches_folder, remaining_images_folder, remaining_scratches, "scratches")
copy_remaining_files_with_structure(no_scratches_folder, remaining_images_folder, remaining_no_scratches, "no_scratches")

print("Files copied successfully with folder structure preserved for both `binclas` and `testdata`!")


Files copied successfully with folder structure preserved for both `binclas` and `testdata`!


In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from tqdm import tqdm
import json
from datetime import datetime
import logging
import sys
import torch
import numpy as np
import torch.nn.functional as F
import open_clip
from torch.optim.lr_scheduler import MultiStepLR
import yaml
import argparse

##################
# Configuration for Binary Classification
##################

# Single category/attribute mapping for binary classification
# "defect_scratch" has 2 classes: no_scratch(0) and scratch(1)
CATEGORY_MAPPING = {
    "defect": {
        "scratch": "class"  # just a placeholder key
    }
}

def load_config(config_path):
    with open(config_path, 'r') as f:
        return yaml.safe_load(f)

def setup_logging(config):
    """Set up logging configuration"""
    os.makedirs(config['paths']['log_dir'], exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_file = os.path.join(config['paths']['log_dir'], f'vith14_binary_class_{timestamp}.log')
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s | %(levelname)s | %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler(sys.stdout)
        ]
    )
    return logging.getLogger(__name__)

##################
# Dataset
##################

class BinaryClassificationDataset(Dataset):
    def __init__(self, root_dir, clip_preprocess_train, clip_preprocess_val, train=True):
        """
        root_dir structure:
         root_dir/
           scratches/
             img_1.jpg
             img_2.jpg
           no_scratches/
             img_a.jpg
             img_b.jpg

        We'll assign:
        - scratch: label=1
        - no_scratch: label=0
        """
        self.root_dir = root_dir
        self.classes = ["no_scratches", "scratches"]  # order defines labeling: no_scratches=0, scratches=1
        self.filepaths = []
        self.labels = []
        self.train = train

        for class_idx, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)
            for fname in os.listdir(class_dir):
                if fname.lower().endswith(('.jpg', '.png', '.jpeg')):
                    self.filepaths.append(os.path.join(class_dir, fname))
                    self.labels.append(class_idx)
        
        self.clip_preprocess_train = clip_preprocess_train
        self.clip_preprocess_val = clip_preprocess_val

    def __len__(self):
        return len(self.filepaths)

    def __getitem__(self, idx):
        image_path = self.filepaths[idx]
        label = self.labels[idx]
        category = "defect"  # a single category for all samples

        image = Image.open(image_path).convert('RGB')
        if self.train:
            image = self.clip_preprocess_train(image)
        else:
            image = self.clip_preprocess_val(image)

        # We have a single attribute "scratch"
        # Our target dict: {"defect_scratch": label}
        targets = {"defect_scratch": label}

        return image, category, targets

##################
# Collate Function
##################
def custom_collate_fn(batch):
    images = torch.stack([item[0] for item in batch])
    categories = [item[1] for item in batch]

    # Single attribute target
    targets = {"defect_scratch": torch.tensor([item[2]["defect_scratch"] for item in batch], dtype=torch.long)}

    return images, categories, targets

##################
# Model
##################

class CategoryAwareAttributePredictor(nn.Module):
    def __init__(self, clip_dim=768, category_attributes=None, attribute_dims=None, hidden_dim=512, dropout_rate=0.2, num_hidden_layers=1):
        super(CategoryAwareAttributePredictor, self).__init__()
        
        self.category_attributes = category_attributes
        self.attribute_predictors = nn.ModuleDict()
        
        for category, attributes in category_attributes.items():
            for attr_name in attributes.keys():
                key = f"{category}_{attr_name}"
                if key in attribute_dims:
                    layers = []
                    # Input layer
                    layers.append(nn.Linear(clip_dim, hidden_dim))
                    layers.append(nn.LayerNorm(hidden_dim))
                    layers.append(nn.ReLU())
                    layers.append(nn.Dropout(dropout_rate))
                    
                    # Additional hidden layers
                    current_dim = hidden_dim
                    for _ in range(num_hidden_layers - 1):
                        layers.append(nn.Linear(current_dim, current_dim // 2))
                        layers.append(nn.LayerNorm(current_dim // 2))
                        layers.append(nn.ReLU())
                        layers.append(nn.Dropout(dropout_rate))
                        current_dim = current_dim // 2

                    # Output layer
                    layers.append(nn.Linear(current_dim, attribute_dims[key]))
                    
                    self.attribute_predictors[key] = nn.Sequential(*layers)
    
    def forward(self, clip_features, category):
        results = {}
        category_attrs = self.category_attributes[category]
        clip_features = clip_features.float()
        
        for attr_name in category_attrs.keys():
            key = f"{category}_{attr_name}"
            if key in self.attribute_predictors:
                results[key] = self.attribute_predictors[key](clip_features)
        
        return results

##################
# Training Function
##################

def train_model(
        clip_model, 
        model, 
        train_dataset,
        train_loader, 
        device,  
        clip_lr,
        predictor_lr, 
        weight_decay, 
        beta1, 
        beta2, 
        hidden_dim, 
        dropout_rate, 
        num_hidden_layers, 
        milestones, 
        gamma, 
        logger, 
        checkpoint_dir,
        num_epochs=10, 
        accumulation_steps=2
    ):

    logger.info("Starting training...")
    criterion = nn.CrossEntropyLoss(ignore_index=-1)

    optimizer = optim.AdamW(
        [
            {'params': clip_model.parameters(), 'lr': clip_lr},
            {'params': model.parameters(), 'lr': predictor_lr, 'weight_decay': weight_decay}
        ], 
        betas=(beta1, beta2)
    )
    scaler = torch.cuda.amp.GradScaler()

    scheduler = MultiStepLR(optimizer=optimizer, milestones=milestones, gamma=gamma)
    metrics_history = {'train_loss': [], 'train_acc': []}
    
    for epoch in range(num_epochs):
        print("here")
        logger.info(f"Epoch {epoch+1}/{num_epochs}")
        model.train()
        clip_model.train()

        train_loss = 0.0
        correct = 0
        total = 0
        num_batches = 0

        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
        # print("abbbtooo?")
        optimizer.zero_grad()

        for batch_idx, (images, categories, targets) in enumerate(pbar):
            # print("or yhaan?")
            images = images.to(device)
            batch_size = images.size(0)

            with torch.cuda.amp.autocast():
                image_features = clip_model.encode_image(images)
                # print("idhar kyaaa?")
                # For each sample we get predictions
                # We have only one attribute: defect_scratch
                predictions = model(image_features, categories[0])  # all belong to "defect"
                pred_logits = predictions["defect_scratch"]  # shape [batch_size, 2]

                target_vals = targets["defect_scratch"].to(device)
                loss = criterion(pred_logits, target_vals)
                loss = loss / accumulation_steps
                # print(loss)

            scaler.scale(loss).backward()

            if (batch_idx + 1) % accumulation_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            # Metrics
            train_loss += loss.item() * accumulation_steps
            num_batches += 1

            _, predicted = torch.max(pred_logits, 1)
            correct += (predicted == target_vals).sum().item()
            total += batch_size

            pbar.set_postfix(loss=(train_loss/num_batches), acc=correct/total)

        scheduler.step()

        avg_loss = train_loss / num_batches
        accuracy = correct / total
        metrics_history['train_loss'].append(avg_loss)
        metrics_history['train_acc'].append(accuracy)

        logger.info(f"Epoch {epoch+1} - Loss: {avg_loss:.4f}, Acc: {accuracy:.4f}")

        # Save a checkpoint each epoch
        os.makedirs(checkpoint_dir, exist_ok=True)
        save_path = os.path.join(checkpoint_dir, f"binary_checkpoint_epoch{epoch+1}.pth")
        torch.save({
            'model_state_dict': model.state_dict(),
            'clip_model_state_dict': clip_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'metrics': metrics_history
        }, save_path)

    logger.info("Training completed.")
    return model, clip_model, metrics_history

##################
# Main
##################
def main():
    # Instead of using argparse, just directly use `config_data`
    config = config_data

    # Set up logging
    logger = setup_logging(config)
    print("helooooooooooooo")
    logger.info("Starting training with provided config")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("imhere22222222")
    # Create CLIP model
    clip_model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms(
        config['model']['name'],
        pretrained=config['model']['pretrained'],
        device=device
    )
    print("aayakyaaa")
    clip_model = clip_model.float()

    # Prepare dataset
    train_dir = os.path.join(config['paths']['data_dir'])
    train_dataset = BinaryClassificationDataset(
        root_dir=train_dir,
        clip_preprocess_train=preprocess_train,
        clip_preprocess_val=preprocess_val,
        train=True
    )
    print("aayakyaa222")

    train_loader = DataLoader(
        train_dataset,
        batch_size=config['training']['batch_size'],
        shuffle=True,
        num_workers=8,
        pin_memory=True,
        collate_fn=custom_collate_fn
    )
    print("haan bhai")
    # Define attribute_dims for binary classification
    attribute_dims = {"defect_scratch": 2}

    model = CategoryAwareAttributePredictor(
        clip_dim=config['model']['clip_dim'],
        category_attributes=CATEGORY_MAPPING,
        attribute_dims=attribute_dims,
        hidden_dim=config['model']['hidden_dim'][0],
        dropout_rate=config['model']['dropout_rate'][0],
        num_hidden_layers=config['model']['num_hidden_layers'][0]
    ).to(device)
    print("abtobol")
    model, clip_model, metrics = train_model(
        clip_model=clip_model,
        model=model,
        train_dataset=train_dataset,
        train_loader=train_loader,
        device=device,
        clip_lr=config['optimizer']['clip_lr'][0],
        predictor_lr=config['optimizer']['predictor_lr'][0],
        weight_decay=config['optimizer']['weight_decay'][0],
        beta1=config['optimizer']['beta1'][0],
        beta2=config['optimizer']['beta2'][0],
        hidden_dim=config['model']['hidden_dim'][0],
        dropout_rate=config['model']['dropout_rate'][0],
        num_hidden_layers=config['model']['num_hidden_layers'][0],
        gamma=config['scheduler']['gamma'],
        milestones=config['scheduler']['milestones'],
        logger=logger,
        checkpoint_dir=config['paths']['checkpoint_dir'],
        num_epochs=config['training']['num_epochs'],
        accumulation_steps=config['training']['accumulation_steps']
    )



In [32]:
main()

helooooooooooooo
imhere22222222


/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/tmp/ipykernel_23/4039194646.py:203: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


aayakyaaa
aayakyaa222
haan bhai
abtobol
here


Epoch 1:   0%|          | 0/1166 [00:00<?, ?it/s]/tmp/ipykernel_23/4039194646.py:228: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1: 100%|██████████| 1166/1166 [01:52<00:00, 10.36it/s, acc=0.927, loss=0.204]


here


...

here


Epoch 4: 100%|██████████| 1166/1166 [01:51<00:00, 10.48it/s, acc=0.988, loss=0.042] 


here


Epoch 5:  34%|███▍      | 397/1166 [00:38<01:14, 10.33it/s, acc=0.992, loss=0.0297]


KeyboardInterrupt: 

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import open_clip
import yaml
import os

##################
# CATEGORY_MAPPING
##################
CATEGORY_MAPPING = {
    "defect": {
        "scratch": "class"
    }
}

##################
# CategoryAwareAttributePredictor
##################
class CategoryAwareAttributePredictor(nn.Module):
    def __init__(self, clip_dim=768, category_attributes=None, attribute_dims=None, hidden_dim=512, dropout_rate=0.2, num_hidden_layers=1):
        super(CategoryAwareAttributePredictor, self).__init__()
        
        self.category_attributes = category_attributes
        self.attribute_predictors = nn.ModuleDict()
        
        for category, attributes in category_attributes.items():
            for attr_name in attributes.keys():
                key = f"{category}_{attr_name}"
                if key in attribute_dims:
                    layers = []
                    # Input layer
                    layers.append(nn.Linear(clip_dim, hidden_dim))
                    layers.append(nn.LayerNorm(hidden_dim))
                    layers.append(nn.ReLU())
                    layers.append(nn.Dropout(dropout_rate))
                    
                    # Additional hidden layers
                    current_dim = hidden_dim
                    for _ in range(num_hidden_layers - 1):
                        layers.append(nn.Linear(current_dim, current_dim // 2))
                        layers.append(nn.LayerNorm(current_dim // 2))
                        layers.append(nn.ReLU())
                        layers.append(nn.Dropout(dropout_rate))
                        current_dim = current_dim // 2

                    # Output layer
                    layers.append(nn.Linear(current_dim, attribute_dims[key]))
                    
                    self.attribute_predictors[key] = nn.Sequential(*layers)
    
    def forward(self, clip_features, category):
        results = {}
        category_attrs = self.category_attributes[category]
        clip_features = clip_features.float()
        
        for attr_name in category_attrs.keys():
            key = f"{category}_{attr_name}"
            if key in self.attribute_predictors:
                results[key] = self.attribute_predictors[key](clip_features)
        
        return results

##################
# Helper Functions
##################

def load_config(config_path):
    with open(config_path, 'r') as f:
        return yaml.safe_load(f)

def load_models(config, checkpoint_path, device):
    # Create CLIP model and transforms
    clip_model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms(
        config['model']['name'],
        pretrained=config['model']['pretrained'],
        device=device
    )
    clip_model = clip_model.float()
    
    # Define attribute_dims (binary classification: 2 classes)
    attribute_dims = {"defect_scratch": 2}
    
    model = CategoryAwareAttributePredictor(
        clip_dim=config['model']['clip_dim'],
        category_attributes=CATEGORY_MAPPING,
        attribute_dims=attribute_dims,
        hidden_dim=config['model']['hidden_dim'][0],
        dropout_rate=config['model']['dropout_rate'][0],
        num_hidden_layers=config['model']['num_hidden_layers'][0]
    ).to(device)

    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    clip_model.load_state_dict(checkpoint['clip_model_state_dict'])

    model.eval()
    clip_model.eval()
    
    return clip_model, model, preprocess_val

def infer_image(clip_model, model, preprocess, image_path, device):
    image = Image.open(image_path).convert('RGB')
    image_tensor = preprocess(image).unsqueeze(0).to(device)

    category = "defect"  # known from training
    
    with torch.no_grad():
        image_features = clip_model.encode_image(image_tensor)
        predictions = model(image_features, category)
        logits = predictions["defect_scratch"]  # shape [1, 2]
        probs = F.softmax(logits, dim=1)
        pred_class = torch.argmax(probs, dim=1).item()

        class_names = ["no_scratches", "scratches"]
        pred_label = class_names[pred_class]
        
        return pred_label, probs.cpu().numpy()

##################
# Example usage (adjust paths as needed)
##################
if __name__ == "__main__":
    # Path to config and checkpoint
    config_path = "/kaggle/working/config.yml"
    checkpoint_path = "/kaggle/working/binary_checkpoint_epoch4.pth"
    test_image_path = "/kaggle/working/testdata/scratches/Code01447.png"

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    config = load_config(config_path)

    # Load models
    clip_model, model, preprocess_val = load_models(config, checkpoint_path, device)

    # Run inference on a test image
    pred_label, probs = infer_image(clip_model, model, preprocess_val, test_image_path, device)
    print(f"Predicted label: {pred_label}, Probabilities: {probs}")

# if __name__ == "__main__":
#     # Example usage:
#     # Provide the path to your config and a checkpoint file
#     config_path = "/kaggle/working/config.yml"
#     checkpoint_path = "/kaggle/working/binary_checkpoint_epoch7.pth"
#     test_image_path = "/kaggle/input/binnary-classification/scratches/03_08_2024_17_12_41.304965_classifier_input.png"

#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     config = load_config(config_path)
#     clip_model, model, preprocess_val = load_models(config, checkpoint_path, device)

#     pred_label, probs = infer_image(clip_model, model, preprocess_val, test_image_path, device)
#     print(f"Predicted label: {pred_label}, Probabilities: {probs}")


/tmp/ipykernel_23/2747002733.py:95: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


Predicted label: scratches, Probabilities: [[0.00258269 0.99741733]]


In [35]:
from sklearn.metrics import precision_recall_fscore_support

def evaluate_model(clip_model, model, preprocess, test_folder, device, num_samples=150):
    class_names = ["no_scratches", "scratches"]
    
    # Collect all test image paths and labels
    test_images = []
    labels = []
    for class_name in class_names:
        class_folder = os.path.join(test_folder, class_name)
        for file_name in os.listdir(class_folder):
            if file_name.endswith(('.png', '.jpg', '.jpeg')):
                test_images.append(os.path.join(class_folder, file_name))
                labels.append(class_name)
    
    # Randomly select a subset of images for evaluation
    selected_indices = random.sample(range(len(test_images)), min(num_samples, len(test_images)))
    selected_images = [test_images[i] for i in selected_indices]
    selected_labels = [labels[i] for i in selected_indices]
    
    # Perform inference on the selected images
    preds = []
    for image_path in selected_images:
        pred_label, _ = infer_image(clip_model, model, preprocess, image_path, device)
        preds.append(pred_label)
    
    # Convert class names to binary labels
    label_map = {name: idx for idx, name in enumerate(class_names)}
    y_true = [label_map[label] for label in selected_labels]
    y_pred = [label_map[pred] for pred in preds]
    
    # Calculate precision, recall, and F1 score for each class
    precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred, average=None, labels=[0, 1])
    
    # precision, recall, f1 now are arrays with metrics for each class in order: class 0, class 1
    # If you want the overall metrics (macro-average), you can also compute them:
    # macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    
    return precision, recall, f1, support, class_names

if __name__ == "__main__":
    config_path = "/kaggle/working/config.yml"
    checkpoint_path = "/kaggle/working/binary_checkpoint_epoch4.pth"
    test_folder = "/kaggle/working/testdata"

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    config = load_config(config_path)

    # Load models
    clip_model, model, preprocess_val = load_models(config, checkpoint_path, device)

    # Evaluate the model
    precision, recall, f1, support, class_names = evaluate_model(clip_model, model, preprocess_val, test_folder, device, num_samples=100)
    
    for i, class_name in enumerate(class_names):
        print(f"Class: {class_name}")
        print(f"  Precision: {precision[i]:.4f}")
        print(f"  Recall:    {recall[i]:.4f}")
        print(f"  F1 Score:  {f1[i]:.4f}")


/tmp/ipykernel_23/2747002733.py:95: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


Class: no_scratches
  Precision: 0.9643
  Recall:    0.9759
  F1 Score:  0.9701
Class: scratches
  Precision: 0.8750
  Recall:    0.8235
  F1 Score:  0.8485


In [4]:
import open_clip

available_models = open_clip.list_pretrained()
for model_name, pretrained_name in available_models:
    print(f"Model: {model_name}, Pretrained: {pretrained_name}")

Class: no_scratches
  Precision: 0.9390
  Recall:    0.9625
  F1 Score:  0.9506
Class: scratches
  Precision: 0.8333
  Recall:    0.7500
  F1 Score:  0.7895

Model: RN50, Pretrained: openai
Model: RN50, Pretrained: yfcc15m
Model: RN50, Pretrained: cc12m
Model: RN101, Pretrained: openai
Model: RN101, Pretrained: yfcc15m
Model: RN50x4, Pretrained: openai
Model: RN50x16, Pretrained: openai
Model: RN50x64, Pretrained: openai
Model: ViT-B-32, Pretrained: openai
Model: ViT-B-32, Pretrained: laion400m_e31
Model: ViT-B-32, Pretrained: laion400m_e32
Model: ViT-B-32, Pretrained: laion2b_e16
Model: ViT-B-32, Pretrained: laion2b_s34b_b79k
Model: ViT-B-32, Pretrained: datacomp_xl_s13b_b90k
Model: ViT-B-32, Pretrained: datacomp_m_s128m_b4k
Model: ViT-B-32, Pretrained: commonpool_m_clip_s128m_b4k
Model: ViT-B-32, Pretrained: commonpool_m_laion_s128m_b4k
Model: ViT-B-32, Pretrained: commonpool_m_image_s128m_b4k
Model: ViT-B-32, Pretrained: commonpool_m_text_s128m_b4k
Model: ViT-B-32, Pretrained: commonpool_m_basic_s128m_b4k
Model: ViT-B-32, Pretrained: commonpool_m_s128m_b4k
Model: ViT-B-32, Pretrained: datacomp_s_s13m_b4k
Model: ViT-B-32, Pretrained: comm